In [1]:
from tensorflow import keras
import random
import numpy as np
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [3]:
all_data = np.concatenate((x_train, x_test))

In [4]:
max_value = float(all_data.max())
al_data_normalized = all_data.astype(np.float32) / max_value
x_train_normalized, x_valid_normalized = al_data_normalized[:-1000], al_data_normalized[-1000:]

In [5]:
all_y = np.concatenate((y_train, y_test))

In [6]:
y_train, y_valid = all_y[:-1000], all_y[-1000:]

In [7]:
x_train = x_train/255.0
x_test = x_test/255.0

In [8]:
coding_size = 30

In [9]:
discriminator = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [10]:
generator = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[coding_size]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(28*28, activation="sigmoid"),
    keras.layers.Reshape([28,28])
])

In [11]:
gan = keras.models.Sequential([generator,discriminator])

In [12]:
discriminator.compile(loss="binary_crossentropy",optimizer="rmsprop")

In [13]:
discriminator.trainable=False

In [14]:
gan.compile(loss="binary_crossentropy",optimizer="rmsprop")

In [15]:
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [16]:
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
  generator, discriminator = gan.layers
  for epoch in range(n_epochs):
    for X_batch in dataset:
      # phase 1 - training the discriminator
      X_batch = tf.cast(X_batch, tf.float32)
      noise = tf.random.normal(shape=[batch_size, codings_size])
      generated_images = generator(noise)
      X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
      y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
      discriminator.trainable = True
      discriminator.train_on_batch(X_fake_and_real, y1)
      # phase 2 - training the generator
      noise = tf.random.normal(shape=[batch_size, codings_size])
      y2 = tf.constant([[1.]] * batch_size)
      discriminator.trainable = False
      gan.train_on_batch(noise, y2)

In [ ]:
train_gan(gan, dataset, batch_size, coding_size)